# Standard CNN Architectures
* https://keras.io/applications/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [8]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.8


In [9]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [10]:
# for VGG, ResNet, and MobileNet
# INPUT_SHAPE = (224, 224)

# for InceptionV3, InceptionResNetV2, Xception
INPUT_SHAPE = (299, 299)

In [11]:
EPOCHS = 50

In [12]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 500

In [13]:
!rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [14]:
!ls -lh

total 7.0M
-rw-rw-r-- 1 ubuntu ubuntu 303K Sep 27 15:22 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rw-rw-r-- 1 ubuntu ubuntu 140K Sep 27 15:22 cnn-augmentation.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.6M Sep 27 15:22 cnn-comparing-all-models.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 380K Sep 27 15:27 cnn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.3M Sep 27 15:22 cnn-prediction.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  13K Sep 28 09:10 cnn-standard-architectures.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 198K Sep 27 15:22 cnn-train-augmented.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  48K Sep 27 15:22 imagenet-pretrained.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 495K Sep 27 15:22 london.jpg
drwxrwxr-x 3 ubuntu ubuntu 4.0K Sep 27 15:25 __MACOSX
-rw-rw-r-- 1 ubuntu ubuntu 127K Sep 27 15:22 Michigan-MSU-raschka.jpg
-rw-rw-r-- 1 ubuntu ubuntu 519K Sep 27 15:22 ml_intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 113K Sep 27 15:22 nn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu   63 Sep 27 15:22 README.html
drwxrwxr-x 8 ubuntu ubuntu 4.0K Sep 27 15:

In [15]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [16]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
X, y = load_data(original_dir, type=".ppm")

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((303, 299, 299, 3), (303, 6))

## VGG

In [ ]:
from keras import applications
# applications.VGG16?
# model = applications.VGG16(include_top=False, weights='imagenet')
model = applications.VGG16(classes=6, weights=None)

In [ ]:
# lacks dropout as this only exists during training

# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# %time model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.3, callbacks=[tb_callback])

## MobileNet

In [ ]:
from keras.applications.mobilenet import MobileNet

model = MobileNet(classes=6, weights=None)

In [ ]:
# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# %time model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.3, callbacks=[tb_callback])

In [ ]:
# train_loss, train_accuracy = model.evaluate(X_train, y_train)
# train_loss, train_accuracy

In [ ]:
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# test_loss, test_accuracy

## ResNet

In [ ]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(classes=6, weights=None)

In [ ]:
# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
!rm -rf tf_log

In [ ]:
# https://github.com/fchollet/keras/issues/6014
# batch normalization seems to mess with accuracy when test data set is small, accuracy here is different from below
%time model.fit(X_train, y_train, epochs=100, validation_split=0.3, callbacks=[tb_callback], batch_size=BATCH_SIZE)

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

## InceptionV3

In [20]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(classes=6, weights=None)

In [21]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [22]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
!rm -rf tf_log

In [27]:
# Depends on harware GPU architecture, model is really complex, batch needs to be small (this works well on K80)
BATCH_SIZE = 10

In [28]:
# https://github.com/fchollet/keras/issues/6014
# batch normalization seems to mess with accuracy when test data set is small, accuracy here is different from below
%time model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[tb_callback], batch_size=BATCH_SIZE)

Train on 212 samples, validate on 91 samples
Epoch 1/100
212/212 [==============================] - 18s - loss: 2.6185 - acc: 0.2217 - val_loss: 1.7948 - val_acc: 0.1429
Epoch 2/100
212/212 [==============================] - 15s - loss: 1.7889 - acc: 0.2311 - val_loss: 2.3642 - val_acc: 0.1429
Epoch 3/100
212/212 [==============================] - 15s - loss: 1.7242 - acc: 0.2689 - val_loss: 1.7779 - val_acc: 0.2527
Epoch 4/100
212/212 [==============================] - 15s - loss: 1.7445 - acc: 0.2500 - val_loss: 2.0354 - val_acc: 0.0879
Epoch 5/100
212/212 [==============================] - 15s - loss: 1.7614 - acc: 0.2500 - val_loss: 12.1730 - val_acc: 0.1429
Epoch 6/100
212/212 [==============================] - 15s - loss: 1.7263 - acc: 0.2877 - val_loss: 2.7359 - val_acc: 0.1429
Epoch 7/100
212/212 [==============================] - 15s - loss: 1.7386 - acc: 0.2264 - val_loss: 7.7407 - val_acc: 0.1429
Epoch 8/100
212/212 [==============================] - 15s - loss: 1.7636 - acc

KeyboardInterrupt: 

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

## Inception-v4, Inception-ResNet

In [19]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

model = InceptionResNetV2(classes=6, weights=None)

AttributeError: module 'keras.applications' has no attribute 'inception_resnet_v2'

In [ ]:
# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
!rm -rf tf_log

In [ ]:
# https://github.com/fchollet/keras/issues/6014
# batch normalization seems to mess with accuracy when test data set is small, accuracy here is different from below
%time model.fit(X_train, y_train, epochs=100, validation_split=0.3, callbacks=[tb_callback], batch_size=BATCH_SIZE)

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy